In [2]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [ ]:
from typing import Optional, List, Dict, Tuple

In [ ]:
from core.models.conv_vqvae import ConvVQMotionModel
from configs.config import cfg, get_cfg_defaults
from core.models.conformer_vqvae import ConformerVQMotionModel

In [324]:
path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml"
cfg = get_cfg_defaults()
print("loading config from:", path)
cfg.merge_from_file(path)
cfg.freeze()

loading config from: /srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml


In [7]:
vqvae_model = ConformerVQMotionModel(
    cfg.vqvae,
    # is_distributed=self.is_distributed,
)

Sync is turned on False


In [21]:
total = sum(p.numel() for p in vqvae_model.motionEncoder.parameters() if p.requires_grad)
print("Total training params: %.2fM" % (total / 1e6))


Total training params: 16.58M


In [10]:
x = torch.randn((1,80,271))

In [13]:
y, ind, lss = vqvae_model(x)

In [14]:
y.shape

torch.Size([1, 80, 271])

In [22]:
cfg.vqvae.depth

6

## Calc mean comman

In [7]:
hml = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D_SMPL"
aist = "/srv/hays-lab/scratch/sanisetty3/music_motion/AIST_SMPL"
cm ="/srv/hays-lab/scratch/sanisetty3/music_motion/Choreomaster_SMPL"

In [11]:
mh = np.load(os.path.join(hml, "Mean.npy"))
ma = np.load(os.path.join(aist, "Mean.npy"))
mc = np.load(os.path.join(cm, "Mean.npy"))

sh = np.load(os.path.join(hml, "Std.npy"))
sa = np.load(os.path.join(aist, "Std.npy"))
sc = np.load(os.path.join(cm, "Std.npy"))


In [13]:
mh.shape

(271,)

In [17]:
m = np.mean([mh, ma,mc] , 0)
s = np.mean([sh,sa,sc] , 0)

In [20]:
np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/pretrained/common/Std.npy" , s)

## Muse stuff

In [31]:
from functools import partial

In [27]:
from dataclasses import dataclass

@dataclass
class AttentionParams:
    dim: int = 768
    dim_head: int = 96
    heads: int = 8
    causal: bool = False
    qk_norm: bool = False
    qk_norm_scale: int = 8
    dropout: float = 0.0
    cross_attn_tokens_dropout: float = (0.0,)
    add_null_kv: bool = False
    flash: bool = False


In [28]:
a = AttentionParams()

In [33]:
a.causal = True

In [38]:
from enum import Enum

class PositionalEmbeddingType(Enum):
    REL= "RelativePositionBias"
    SINE= "ScaledSinusoidalEmbedding"
    ALIBI= "AlibiPositionalBias"
    ABS= "AbsolutePositionalEmbedding"
    SHAW= "ShawRelativePositionalEmbedding"


In [40]:
a = PositionalEmbeddingType.REL

In [68]:
a = torch.randn((4,100,768))

In [48]:
norm = nn.InstanceNorm1d(768)

In [51]:
def uniform(shape, min=0, max=1, device=None):
    return torch.zeros(shape, device=device).float().uniform_(0, 1)


def prob_mask_like(shape, prob, device=None):
    if prob == 1:
        return torch.ones(shape, device=device, dtype=torch.bool)
    elif prob == 0:
        return torch.zeros(shape, device=device, dtype=torch.bool)
    else:
        return uniform(shape, device=device) < prob


In [53]:
b,n,d = a.shape

In [60]:
mask_ = prob_mask_like((b, 1), 1.0 - 0.5, "cpu")


In [61]:
mask_

tensor([[False],
        [ True],
        [False],
        [False]])

In [69]:
context_mask = (a != 1026).any(dim=-1)

In [71]:
mask_

tensor([[False],
        [ True],
        [False],
        [False]])

In [63]:
context_mask = context_mask & mask_

In [7]:
from transformers import T5Config, T5EncoderModel, T5Tokenizer


/srv/share2/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
config = T5Config.from_pretrained("google/t5-v1_1-base")

In [328]:
from core.models.motion_clip.motion_clip import MotionClipEncoder

ckpt = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/core/models/motion_clip/MotionClipEncoder.pt", map_location="cpu")
enc = MotionClipEncoder()
enc.load_state_dict(ckpt)

<All keys matched successfully>

In [49]:
y = torch.zeros(10, dtype=int)

In [50]:
y = y-y

In [59]:
y.shape

torch.Size([10])

In [1]:
cd ../MotionCLIP/

/coc/scratch/sanisetty3/music_motion/MotionCLIP


In [2]:
from src.parser.visualize import parser


In [4]:
parameters= parser()

In [5]:
parameters

{'activation': 'gelu',
 'align_pose_frontview': False,
 'archiname': 'transformer',
 'batch_size': 20,
 'clip_image_losses': ['cosine'],
 'clip_lambda_ce': 1.0,
 'clip_lambda_cosine': 1.0,
 'clip_lambda_mse': 1.0,
 'clip_lambdas': {'image': {'cosine': 1.0}, 'text': {'cosine': 1.0}},
 'clip_map_images': False,
 'clip_map_text': False,
 'clip_mappers_type': 'no_mapper',
 'clip_text_losses': ['cosine'],
 'cuda': True,
 'datapath': './data/amass_db/amass_30fps_db.pt',
 'dataset': 'amass',
 'debug': False,
 'device': device(type='cuda', index=0),
 'expname': 'exps',
 'folder': './exps/paper-model',
 'glob': True,
 'glob_rot': [3.141592653589793, 0, 0],
 'jointstype': 'smpl',
 'lambda_kl': 0.0,
 'lambda_rc': 100.0,
 'lambda_rcxyz': 100.0,
 'lambda_vel': 100.0,
 'lambda_velxyz': 1.0,
 'lambdas': {'rc': 100.0, 'rcxyz': 100.0, 'vel': 100.0},
 'latent_dim': 512,
 'losses': ['rc', 'rcxyz', 'vel'],
 'lr': 0.0001,
 'max_len': -1,
 'min_len': -1,
 'modelname': 'cvae_transformer_rc_rcxyz_vel',
 'mode

In [330]:
import smplx
import pickle

In [18]:
npz = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/motion_vqvae/body_models/smplh/neutral/model.npz")

In [78]:
npz.files

['J_regressor_prior',
 'f',
 'J_regressor',
 'kintree_table',
 'J',
 'weights_prior',
 'weights',
 'posedirs',
 'bs_style',
 'v_template',
 'shapedirs',
 'bs_type']

In [19]:
data = {}
for fn in npz.files:
    data[fn] = npz[fn]

In [23]:
import pickle
with open(smpl_p, 'rb') as f:
    sm = pickle.load(f, encoding='latin1')

/tmp/ipykernel_1873241/1164358967.py:3: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  sm = pickle.load(f, encoding='latin1')


In [22]:
# with open("/srv/hays-lab/scratch/sanisetty3/music_motion/motion_vqvae/body_models/smplh/" + "SMPLH_NEUTRAL.pkl", 'wb') as handle:
#     sm = pickle.dump(data, handle, encoding = 'latin1')

In [11]:
# smplh = smplx.SMPLH(path + "SMPLH_NEUTRAL.pkl")

In [26]:
np.savez("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/body_models/smpl/smpl_neutral.npz" ,sm, allow_pickle=True)

In [5]:
import clip
model, preprocess = clip.load('ViT-B/32', device)

(219, 72)

In [339]:
(db["joints3d"])[0].shape

(219, 18, 3)

In [360]:
ret.shape

torch.Size([219, 144])

In [368]:
import joblib

db_file = "/srv/hays-lab/scratch/sanisetty3/music_motion/MotionCLIP/data/amass_db/amass_30fps_test.pt"
db = joblib.load(db_file)
db["thetas"][0].shape

ret = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(torch.Tensor(db["thetas"][0]).reshape(-1,24,3))).reshape(-1,144)

joints3D =(db["joints3d"])[0]
joints3D = joints3D - joints3D[0, 0, :]
ret_tr = torch.Tensor(joints3D)[:, 0, :]
              
ret_tr = torch.Tensor(ret_tr - ret_tr[0])
padded_tr = torch.zeros((ret.shape[0], 6), dtype=ret.dtype)
padded_tr[:, :3] = ret_tr
ret2 = torch.cat((ret, padded_tr), 1)

In [363]:
ret2.shape

torch.Size([219, 150])

In [369]:
from human_body_prior.tools.omni_tools import copy2cpu as c2c
from human_body_prior.body_model.body_model import BodyModel
from utils import rotation_conversions as geometry


In [57]:
path = "/srv/hays-lab/scratch/sanisetty3/music_motion/motion_vqvae/body_models/smplh/neutral/model.npz"
smpl_p = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/body_models/smpl/smpl_neutral.npz"

In [61]:
smplh = BodyModel(path)

In [370]:
def aist_axis_angle_from_6d(smpl_motion: torch.Tensor) -> torch.Tensor:
    trans = smpl_motion[:, :3]
    rots = smpl_motion[:, 3:]

    aa_rots = geometry.matrix_to_axis_angle(geometry.rotation_6d_to_matrix(rots.reshape(-1, 22, 6))).reshape(rots.shape[0], -1)
    smpl_aa = torch.cat([trans, aa_rots], 1)

    return smpl_aa

In [371]:
def process_for_clip(motion):
    trans = motion[:, :3]
    trans = torch.cat([trans , torch.zeros(trans.shape[0], 3)] , 1)
    rots = motion[:, 3:]
    rots = torch.cat([rots , torch.zeros(rots.shape[0], 12)] , 1)
    
    new_mot = torch.cat([rots , trans] , 1)
    new_mot = new_mot.reshape(1, trans.shape[0] , 25, 6).permute(0,2,3,1)
    return new_mot

In [372]:
from enum import Enum
AIST_GENRE = {"GBR" : "Break",
    "GPO" : "Pop",
    "GLO" : "Lock",
    "GMH" : "Middle Hip-hop",
    "GLH" : "LA style Hip-hop",
    "GHO" : "House",
    "GWA" : "Waack",
    "GKR" : "Krump",
    "GJS" : "Street Jazz",
    "GJB" : "Ballet Jazz"}


In [188]:
text_inputs = torch.cat([clip.tokenize(f"{c} dance style") for c in AIST_GENRE.values()]).to(device)
text_features = model.encode_text(text_inputs)
text_features /= text_features.norm(dim=-1, keepdim=True)


In [199]:
motion = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/00_000000.npy")
motion = torch.Tensor(motion)[:,:135]

In [209]:
emb = enc.encode_motions(process_for_clip(motion)[...,:60]).to(device)
emb /= emb.norm(dim=-1, keepdim=True)

text_inputs = torch.cat([clip.tokenize(f"{c} dance style") for c in AIST_GENRE.values()]).to(device)
text_features = model.encode_text(text_inputs)
text_features /= text_features.norm(dim=-1, keepdim=True)

similarity = (100.0 * emb @ text_features.float().T).softmax(dim=-1)
values, indices = similarity[0].topk(10)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{list(AIST_GENRE.values())[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

             Pop: 37.20%
  Middle Hip-hop: 28.40%
           House: 14.33%
            Lock: 7.38%
           Krump: 6.23%
LA style Hip-hop: 3.05%
     Street Jazz: 1.50%
           Waack: 0.83%
     Ballet Jazz: 0.82%
           Break: 0.27%


In [10]:
from glob import glob
import codecs as cs
from scipy.spatial.transform import Rotation as R

In [246]:
def rotate_smpl_by(input_smpl_6d, y_rot):
    if isinstance(y_rot, list):
        angl = y_rot
    else:
        angl = [0,y_rot,0]
        
    input_smpl = torch.Tensor(input_smpl_6d)
    r = R.from_euler('XYZ', angl, degrees=True)
    r_matrix = torch.Tensor(r.as_matrix())
    r_quat = torch.Tensor(r.as_quat()[[3,0,1,2]])
    
    
    trans = input_smpl[:,:3]
    
    root_orient = input_smpl[:,3:9]
    root_orient_quat = geometry.matrix_to_quaternion(geometry.rotation_6d_to_matrix(root_orient)).reshape(-1,4)
    rotated_all_root_quat = geometry.quaternion_multiply(r_quat , root_orient_quat)
    
    
    input_smpl[:,3:9] = geometry.matrix_to_rotation_6d(geometry.quaternion_to_matrix(rotated_all_root_quat))
    
    input_smpl[:,:3] = trans@r_matrix.T

    
    return input_smpl

In [249]:
nm.shape

torch.Size([116, 135])

In [388]:
texts = glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/texts/*")

In [11]:
def get_caption(path):
    text_data = []
    captions = []
    flag = False
    with cs.open(path) as f:
        for line in f.readlines():
            text_dict = {}
            line_split = line.strip().split('#')
            caption = line_split[0]
            captions.append(caption)
            tokens = line_split[1].split(' ')
            f_tag = float(line_split[2])
            to_tag = float(line_split[3])
            f_tag = 0.0 if np.isnan(f_tag) else f_tag
            to_tag = 0.0 if np.isnan(to_tag) else to_tag
    
            text_dict['caption'] = caption
            text_dict['tokens'] = tokens
            if f_tag == 0.0 and to_tag == 0.0:
                flag = True
                text_data.append(text_dict)
            else:
                try:
                    n_motion = motion[int(f_tag*fps) : int(to_tag*fps)]
                    if (len(n_motion)) < min_motion_len or (len(n_motion) >= 200):
                        continue
                    new_name = random.choice('ABCDEFGHIJKLMNOPQRSTUVW') + '_' + name
                    while new_name in data_dict:
                        new_name = random.choice('ABCDEFGHIJKLMNOPQRSTUVW') + '_' + name
                    data_dict[new_name] = {'motion': n_motion,
                                           'length': len(n_motion),
                                           'text':[text_dict]}
                    new_name_list.append(new_name)
                    length_list.append(len(n_motion))
                except:
                    print(line_split)
                    print(line_split[2], line_split[3], f_tag, to_tag, name)
                    # break
    return captions

In [12]:
captions = get_caption( "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/texts/000000.txt")
captions_opp = get_caption("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/texts/001000.txt")

In [13]:
captions

['a man kicks something or someone with his left leg.',
 'the standing person kicks with their left foot before going back to their original stance.',
 'a man kicks with something or someone with his left leg.',
 'he is flying kick with his left leg']

In [276]:
all_caps = captions + captions_opp 

In [302]:
motion = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/00_000000.npy")
motion = torch.Tensor(motion)[:,:135]
nm = rotate_smpl_by(motion, [90,-90,0])

motion1 = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/180_000000.npy")
motion1 = torch.Tensor(motion1)[:,:135]
nm1 = rotate_smpl_by(motion1, [90,-90,0])

motion2 = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/00_001000.npy")
motion2 = torch.Tensor(motion2)[:,:135]
nm2 = rotate_smpl_by(motion2, [90,-90,0])

motion3 = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/000000_og.npy")
motion3 = torch.Tensor(motion3)[:,:135]
nm3 = rotate_smpl_by(motion3, [90,-90,0])

In [312]:
mms = torch.cat([process_for_clip(motion)[...,:60] , process_for_clip(motion1)[...,:60], process_for_clip(motion2)[...,:60], process_for_clip(motion3)[...,:60]] , 0)
emb = enc.encode_motions((mms)).to(device)
text_inputs = torch.cat([clip.tokenize(c) for c in all_caps]).to(device)
text_features = model.encode_text(text_inputs).float()

emb /= emb.norm(dim=-1, keepdim=True)

text_features /= text_features.norm(dim=-1, keepdim=True)

features_norm = text_features / text_features.norm(dim=-1, keepdim=True)
seq_motion_features_norm = emb / emb.norm(dim=-1, keepdim=True)
logit_scale = model.logit_scale.exp()
logits_per_motion = logit_scale * seq_motion_features_norm @ features_norm.t()
logits_per_d = logits_per_motion.t()

In [304]:
emb /= emb.norm(dim=-1, keepdim=True)

text_features /= text_features.norm(dim=-1, keepdim=True)


In [313]:
features_norm = text_features / text_features.norm(dim=-1, keepdim=True)
seq_motion_features_norm = emb / emb.norm(dim=-1, keepdim=True)
logit_scale = model.logit_scale.exp()
logits_per_motion = logit_scale * seq_motion_features_norm @ features_norm.t()
logits_per_d = logits_per_motion.t()

In [314]:
logits_per_motion

tensor([[68.5927, 66.7999, 68.5085, 66.7495, 73.3126, 70.6648, 70.4461],
        [68.4425, 66.9240, 68.2853, 66.9656, 73.3869, 70.6454, 70.3920],
        [67.5542, 67.4931, 67.5023, 65.7932, 73.7883, 70.8619, 71.4676],
        [67.7534, 66.2508, 67.4441, 65.6629, 72.3491, 69.3443, 69.6399]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [315]:
similarity = logits_per_motion.softmax(dim=-1)


In [316]:
values, indices = similarity[0].topk(len(all_caps))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{all_caps[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

a person does a long step to the right.: 87.13%
someone taking a forward-right step in , leading with their right foot.: 6.17%
person takes a step diagonally forward and to the right with their right foot followed by their left.: 4.96%
a man kicks something or someone with his left leg.: 0.78%
a man kicks with something or someone with his left leg.: 0.71%
the standing person kicks with their left foot before going back to their original stance.: 0.13%
he is flying kick with his left leg: 0.12%


In [307]:
values, indices = similarity[1].topk(len(all_caps))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{all_caps[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

a person does a long step to the right.: 86.50%
someone taking a forward-right step in , leading with their right foot.: 5.95%
person takes a step diagonally forward and to the right with their right foot followed by their left.: 5.44%
a man kicks something or someone with his left leg.: 0.97%
a man kicks with something or someone with his left leg.: 0.80%
he is flying kick with his left leg: 0.19%
the standing person kicks with their left foot before going back to their original stance.: 0.15%


In [308]:
values, indices = similarity[3].topk(len(all_caps))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{all_caps[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

a person does a long step to the right.: 88.85%
person takes a step diagonally forward and to the right with their right foot followed by their left.: 4.84%
someone taking a forward-right step in , leading with their right foot.: 4.72%
a man kicks something or someone with his left leg.: 0.81%
a man kicks with something or someone with his left leg.: 0.53%
the standing person kicks with their left foot before going back to their original stance.: 0.13%
he is flying kick with his left leg: 0.11%


## Do motionclip

In [420]:
from core.models.motion_clip.motion_clip import MotionClipEncoder

ckpt = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/core/models/motion_clip/MotionClipEncoder.pt", map_location="cpu")
enc = MotionClipEncoder()
enc.load_state_dict(ckpt)

<All keys matched successfully>

In [385]:
import clip
model, preprocess = clip.load('ViT-B/32', device)

In [402]:
def process_for_clip(motion):

    if motion.shape[-1] == 135:
        trans = motion[:, :3]
        trans = torch.cat([trans , torch.zeros(trans.shape[0], 3)] , 1)
        rots = motion[:, 3:]
        rots = torch.cat([rots , torch.zeros(rots.shape[0], 12)] , 1)
        
        new_mot = torch.cat([rots , trans] , 1)
    new_mot = new_mot.reshape(1, motion.shape[0] , 25, 6).permute(0,2,3,1)
    return new_mot

In [375]:
import joblib

db_file = "/srv/hays-lab/scratch/sanisetty3/music_motion/MotionCLIP/data/amass_db/amass_30fps_test.pt"
db = joblib.load(db_file)

In [376]:
db.keys()

dict_keys(['vid_names', 'thetas', 'joints3d', 'clip_images', 'clip_pathes', 'text_raw_labels', 'text_proc_labels', 'action_cat'])

In [395]:
def getMotion(idx):
    ret = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(torch.Tensor(db["thetas"][idx]).reshape(-1,24,3))).reshape(-1,144)
    
    joints3D =(db["joints3d"])[idx]
    joints3D = joints3D - joints3D[0, 0, :]
    ret_tr = torch.Tensor(joints3D)[:, 0, :]
                  
    ret_tr = torch.Tensor(ret_tr - ret_tr[0])
    padded_tr = torch.zeros((ret.shape[0], 6), dtype=ret.dtype)
    padded_tr[:, :3] = ret_tr
    ret2 = torch.cat((ret, padded_tr), 1)

    return ret2

In [397]:
motion_og = getMotion(0)

In [396]:


motionn = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/new_joint_vecs/00_001612.npy")
motionn = torch.Tensor(motionn)[:,:135]

In [393]:
captions = get_caption( "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/texts/001612.txt")
captions_opp = get_caption("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/HumanML3D_SMPL/texts/001000.txt")

In [439]:
all_caps = captions + captions_opp + ["jumping jack"]

In [404]:
motions = [motionn , motion_og]

In [440]:
mms = torch.cat([process_for_clip(motionn)[...,:60] for motion in motions] , 0)


In [441]:
emb = enc.encode_motions(mms).to(device)


In [442]:
emb /= emb.norm(dim=-1, keepdim=True)
text_inputs = torch.cat([clip.tokenize(c) for c in all_caps]).to(device)
text_features = model.encode_text(text_inputs).float()
text_features /= text_features.norm(dim=-1, keepdim=True)

similarity = (100.0 * emb @ text_features.float().T).softmax(dim=-1)

In [443]:
similarity

tensor([[0.0071, 0.0129, 0.1549, 0.6157, 0.0876, 0.0816, 0.0402],
        [0.0079, 0.0145, 0.1718, 0.5888, 0.0904, 0.0733, 0.0533]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [444]:
values, indices = similarity[0].topk(len(all_caps))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{all_caps[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

a person does a long step to the right.: 61.57%
a person does jumping jacks followed by a twist.: 15.49%
someone taking a forward-right step in , leading with their right foot.: 8.76%
person takes a step diagonally forward and to the right with their right foot followed by their left.: 8.16%
    jumping jack: 4.02%
person does jumping jacks and does a turn around hop at the end.: 1.29%
doing exercizes with jumping jacks: 0.71%


In [445]:
values, indices = similarity[1].topk(len(all_caps))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{all_caps[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

a person does a long step to the right.: 58.88%
a person does jumping jacks followed by a twist.: 17.18%
someone taking a forward-right step in , leading with their right foot.: 9.04%
person takes a step diagonally forward and to the right with their right foot followed by their left.: 7.33%
    jumping jack: 5.33%
person does jumping jacks and does a turn around hop at the end.: 1.45%
doing exercizes with jumping jacks: 0.79%
